# Figure 13

FALTA ESTO!!!!

In [5]:
import sys
sys.path.append("../src")

import warnings
warnings.filterwarnings("ignore")

import PIL
import numpy as np
import pandas as pd
import seaborn as sns
from io import BytesIO
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from PIL import Image
from pathlib import Path
from statsmodels.formula.api import ols
from pandas.api.types import CategoricalDtype

from Config.config import PATHS
from Utils.indices import AlternationIndex
from Utils.utils import GetMeasurements, Grid
from Utils.plot_utils import PlotsAndMeasures, BarRenderer

In [6]:
file_name = 'multi-player.csv'
file = PATHS['human_data'] / Path(file_name)
print(f'Loading data from {file}...')
data = pd.read_csv(file)
data['source'] = 'humans'
data.drop(columns=['approx_threshold', 'room', 'date', 'timeup', 'time', 'Unnamed: 0'], inplace=True)
data.sort_values(by=['num_players', 'threshold'], inplace=True)
data = data[data['threshold'] != 1]
data['num_players'] = data['num_players'].astype(int)
data.head(2)

Loading data from /Users/edgar/Documents/GitHub/Cognitive_Models_El_Farol_Bar_Problem/data/human/multi-player.csv...


,treatment,threshold,round,player,decision,score,num_players,group,choice,source
7290,upwards,0.333333,1.0,406360458182849,1.0,-1.0,3,17,1.0,humans
7291,upwards,0.333333,1.0,679877000681823,1.0,-1.0,3,17,1.0,humans


In [7]:
from collections import defaultdict

group_counter = {grp:0 for grp in data['group'].unique()}

dict_sessions_downwards = defaultdict()
dict_sessions_upwards = defaultdict()
data.sort_values(by=['group', 'treatment', 'threshold'], inplace=True)
for (group, treatment, threshold), grp in data.groupby(['group', 'treatment', 'threshold']):
    if treatment == 'downwards':
        group_counter[group] += 1
        count = 4 - group_counter[group]
        dict_sessions_downwards[(group, threshold)] = count
    elif treatment == 'upwards':
        group_counter[group] += 1
        count = group_counter[group]
        dict_sessions_upwards[(group, threshold)] = count

dict_sessions = dict_sessions_downwards.copy()
dict_sessions.update(dict_sessions_upwards)

data['session'] = data.apply(lambda row: dict_sessions[(row['group'], row['threshold'])], axis=1)
data['round'] = data['round'].apply(lambda x: (x - 1) % 30)
data.head(2)

,treatment,threshold,round,player,decision,score,num_players,group,choice,source,session
8910,upwards,0.25,0.0,841658927890285,0.0,0.0,4,5,0.0,humans,1
8911,upwards,0.25,0.0,373455036275630,1.0,-1.0,4,5,1.0,humans,1


### Efficiency and alternation index

In [8]:
# Calculate alternation index per group
gm = GetMeasurements(
    data=data,
    measures=['efficiency', 'bounded_efficiency', 'entropy', 'conditional_entropy', 'inequality'],
    T=30
)
df = gm.get_measurements()

alt_index = AlternationIndex.from_file(priority='sklearn')
df['alternation_index'] = alt_index(df)

df.head(2)

AssertionError: round_=15.0
      treatment  threshold  round           player  decision  score  \
8280  downwards   0.333333   15.0  670385314115779       1.0    1.0   
8281  downwards   0.333333   15.0  172850836295148       0.0    0.0   
8282  downwards   0.333333   15.0  277565140184927       0.0    0.0   
8370  downwards   0.333333   15.0  670385314115779       0.0    0.0   
8371  downwards   0.333333   15.0  172850836295148       0.0    0.0   
8372  downwards   0.333333   15.0  277565140184927       1.0    1.0   

      num_players  group  choice  source  session  
8280            3     30     1.0  humans        3  
8281            3     30     0.0  humans        3  
8282            3     30     0.0  humans        3  
8370            3     30     0.0  humans        3  
8371            3     30     0.0  humans        3  
8372            3     30     1.0  humans        3  

### Best alternating groups in the upwards treatment

In [ ]:
df[df['treatment'] == 'upwards'].sort_values(by=['alternation_index'], ascending=False).head(2)

### Best alternating groups in the downwards treatment

In [ ]:
df[df['treatment'] == 'downwards'].sort_values(by=['alternation_index'], ascending=False).head(2)

### Playout examples

In [ ]:
#-------------------------------
# Select groups
#-------------------------------
# groups = data['group'].unique().tolist()
groups = [11, 30] # downwards
#-------------------------------
# Obtain number of bar playouts
#-------------------------------
dfA = data[data['group'].isin(groups)].sort_values(by=['group', 'threshold'], ascending=False)
num_playouts = 0
for key, grp in dfA.groupby(['group', 'threshold']):
    num_playouts += 1
#-------------------------------
# Plot groups
#-------------------------------
grid = Grid(num_playouts, vertical=False)
fig, axes = plt.subplots(
    grid.rows, grid.cols, 
    figsize=(grid.cols*4, grid.rows*1.5),
    tight_layout=True
)

counter = 0
for group in groups:
    grp = dfA[dfA['group'] == group]
    num_pl = grp['num_players'].unique()[0]
    treatment = grp['treatment'].unique()[0]
    for threshold in grp['threshold'].unique():
        grp_thres = grp[grp['threshold'] == threshold]
        # print(f"{grp_thres['num_players'].unique()[0]} players in group {group} with threshold {threshold}")
        # print(grid.index_to_pair(counter))
        bar_renderer = BarRenderer(data=grp_thres)
        bar_renderer.render(
            ax=axes[grid.index_to_pair(counter)], 
            title=f"Treat: {treatment} -- Grp. {group} -- N={grp_thres['num_players'].unique()[0]}  -- B={int(threshold * num_pl)}",
            # title=f"Group ({i+1})",
            num_rounds=30
        )
        counter += 1

for i in range(counter):
    axes[grid.index_to_pair(counter)].set_axis_off()

print(f"Total playouts: {counter}")

# Convert figure to image array
fig.suptitle("A", fontweight="bold")
fig.canvas.draw()
rgba_bytes = fig.canvas.buffer_rgba().tobytes()
width, height = fig.canvas.get_width_height()
pil_image = PIL.Image.frombytes(mode="RGBA", size=[width, height], data=rgba_bytes)
panel_A = np.array(pil_image)

In [ ]:
#-------------------------------
# Select groups
#-------------------------------
# groups = data['group'].unique().tolist()
groups = [33, 15] # upwards
#-------------------------------
# Obtain number of bar playouts
#-------------------------------
dfA = data[data['group'].isin(groups)].sort_values(by=['group', 'threshold'], ascending=True)
num_playouts = 0
for key, grp in dfA.groupby(['group', 'threshold']):
    num_playouts += 1
#-------------------------------
# Plot groups
#-------------------------------
grid = Grid(num_playouts, vertical=False)
fig, axes = plt.subplots(
    grid.rows, grid.cols, 
    figsize=(grid.cols*4, grid.rows*1.5),
    tight_layout=True
)

counter = 0
for group in groups:
    grp = dfA[dfA['group'] == group]
    num_pl = grp['num_players'].unique()[0]
    treatment = grp['treatment'].unique()[0]
    for threshold in grp['threshold'].unique():
        grp_thres = grp[grp['threshold'] == threshold]
        # print(f"{grp_thres['num_players'].unique()[0]} players in group {group} with threshold {threshold}")
        # print(grid.index_to_pair(counter))
        bar_renderer = BarRenderer(data=grp_thres)
        bar_renderer.render(
            ax=axes[grid.index_to_pair(counter)], 
            title=f"Treat: {treatment} -- Grp. {group} -- N={grp_thres['num_players'].unique()[0]}  -- B={int(threshold * num_pl)}",
            # title=f"Group ({i+1})",
            num_rounds=30
        )
        counter += 1
print(f"Total playouts: {counter}")

# Convert figure to image array
fig.suptitle("B", fontweight="bold")
fig.canvas.draw()
rgba_bytes = fig.canvas.buffer_rgba().tobytes()
width, height = fig.canvas.get_width_height()
pil_image = PIL.Image.frombytes(mode="RGBA", size=[width, height], data=rgba_bytes)
panel_B = np.array(pil_image)

### Panels A and B

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(8,4), tight_layout=True)

ax[0].imshow(panel_A)  
ax[1].imshow(panel_B)

ax[0].axis('off')
ax[1].axis('off')

# Convert figure to image array
fig.canvas.draw()
rgba_bytes = fig.canvas.buffer_rgba().tobytes()
width, height = fig.canvas.get_width_height()
pil_image = PIL.Image.frombytes(mode="RGBA", size=[width, height], data=rgba_bytes)
panels_A_and_B = np.array(pil_image)

### Average score per round

In [ ]:
fig, ax = plt.subplots(
    2,1, 
    figsize=(3,5), 
    tight_layout=True,
    sharey=False
)

# Learning: average score increases per round
sns.lineplot(
    x='round', y='score', data=data,
    ci=95, ax=ax[0]
)
ax[0].set_xlabel('Round')
ax[0].set_ylabel("Av. player score")
ax[0].set_title("Learning within session")
ax[0].grid()
ax[0].set_ylim([-1,1])

# Learning: average score increases per session
sns.boxplot(
    x='session', y='score', 
    data=data, ax=ax[1],
)
ax[1].set_xlabel('Session')
ax[1].set_ylabel("Av. player score")
ax[1].set_title("Learning per session")
ax[1].grid()
# ax[1].set_ylim([-1,1])

# Convert figure to image array
fig.suptitle("C", fontweight="bold")
fig.canvas.draw()
rgba_bytes = fig.canvas.buffer_rgba().tobytes()
width, height = fig.canvas.get_width_height()
pil_image = PIL.Image.frombytes(mode="RGBA", size=[width, height], data=rgba_bytes)
panel_C = np.array(pil_image)

Pearson round vs score

In [ ]:
pearson_coef, p_value = stats.pearsonr(data['score'], data['round'])

print(f"Pearson correlation coefficient: {pearson_coef}")
print(f"P-value: {p_value}")
print(f"{pearson_coef:.3f} (p={p_value:.3f})")

Difference in means between session

In [ ]:
# Get the unique session labels
sessions = data['session'].unique()

# Extract the scores for each session
scores_by_session = [data[data['session'] == session]['score'] for session in sessions]

# Perform the Kruskal-Wallis H-test
h_stat, p_value = stats.kruskal(*scores_by_session)

print(f"Kruskal-Wallis H statistic: {h_stat}")
print(f"P-value: {p_value}")
print(f"H={h_stat:.3f} (p={p_value:.3f})")

### bounded efficiency vs. inequality

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4,4), tight_layout=True)
# Scatter plot bounded efficiency vs. inequality
df_jitter = df.copy()
n = df_jitter.shape[0]
df_jitter['inequality'] = df_jitter['inequality'] + np.random.normal(0,0.0075, n)
df_jitter['bounded_efficiency'] = df_jitter['bounded_efficiency'] + np.random.normal(0,0.0075, n)
ax = sns.scatterplot(
    x='inequality', y='bounded_efficiency',
    data=df_jitter,
    # hue='treatment', style='num_players',
    alpha=0.8, ax=ax,
    palette='icefire'
)

ax.set_xlabel('Inequality')
ax.set_ylabel('Bounded Efficiency')
ax.set_title('Efficiency vs. Inequality')
ax.grid()

# Show the alternating groups on the top-left corner
circle = patches.Ellipse(
    (0.15, 0.82), width=0.2, height=0.2,
    transform=ax.transAxes, 
    color='gray', alpha=0.2, 
)
ax.add_patch(circle)
# Add an arrow pointing to the cluster
ax.annotate(
    'Alternating\n groups',
    xy=(0.15, 0.8), xycoords='axes fraction',
    xytext=(0.35, 0.3), textcoords='axes fraction',
    arrowprops=dict(facecolor='black', arrowstyle='->'),
    fontsize=10
)

# Show the mixed groups on the top center
circle = patches.Ellipse(
    (0.55, 0.88), width=0.4, height=0.2,
    transform=ax.transAxes, 
    color='gray', alpha=0.2, 
)
ax.add_patch(circle)
# Add an arrow pointing to the cluster
ax.annotate(
    'Mixed\n groups',
    xy=(0.5, 0.8), xycoords='axes fraction',
    xytext=(0.6, 0.4), textcoords='axes fraction',
    arrowprops=dict(facecolor='black', arrowstyle='->'),
    fontsize=10
)

# Show the segmented groups on the top-right corner
circle = patches.Circle(
    (0.95, 0.95), 0.05, 
    transform=ax.transAxes, 
    color='gray', alpha=0.2
)
ax.add_patch(circle)
# Add an arrow pointing to the cluster
ax.annotate(
    'Segmented\n   group',
    xy=(0.92, 0.93), xycoords='axes fraction',
    xytext=(0.65, 0.1), textcoords='axes fraction',
    arrowprops=dict(facecolor='black', arrowstyle='->'),
    fontsize=10
)

# Convert figure to image array
fig.suptitle("D", fontweight="bold")
fig.canvas.draw()
rgba_bytes = fig.canvas.buffer_rgba().tobytes()
width, height = fig.canvas.get_width_height()
pil_image = PIL.Image.frombytes(mode="RGBA", size=[width, height], data=rgba_bytes)
panel_D = np.array(pil_image)


### Panels C and D

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(7,4), tight_layout=True)

ax[0].imshow(panel_C)  
ax[1].imshow(panel_D)

ax[0].axis('off')
ax[1].axis('off')

# Convert figure to image array
fig.canvas.draw()
rgba_bytes = fig.canvas.buffer_rgba().tobytes()
width, height = fig.canvas.get_width_height()
pil_image = PIL.Image.frombytes(mode="RGBA", size=[width, height], data=rgba_bytes)
panels_C_and_D = np.array(pil_image)

### Panel E --- Influence of threshold and number of players

In [ ]:
fig, ax = plt.subplots(
    2, 2, 
    figsize=(8,6), 
    tight_layout=True,
    # sharex=True, sharey=True
)

# Efficiency per threshold and treatment
sns.lineplot(
    x='threshold', y='bounded_efficiency',
    hue='treatment',
    data=df, ax=ax[0,0],
    palette='icefire'
)
ax[0,0].set_xlabel("Bar's threshold")
ax[0,0].set_ylabel("Av. bounded efficiency")
ax[0,0].set_title("Bounded Eff. per threshold")
ax[0,0].grid()
ax[0,0].set_ylim([-1.6,1.1])
ax[0,0].legend_.remove()

# Efficiency per number of agents and treatment
sns.lineplot(
    x='num_players', y='bounded_efficiency',
    hue='treatment',
    data=df, ax=ax[0,1],
    palette='icefire'
)
ax[0,1].set_xlabel("Number of players")
ax[0,1].set_ylabel("Av. bounded efficiency")
ax[0,1].set_title("Bounded Eff. per Num. Players")
ax[0,1].grid()
ax[0,1].set_ylim([-1.6,1.1])
ax[0,1].legend_.remove()

# Alternation index per threshold and treatment
sns.lineplot(
    x='threshold', y='alternation_index',
    hue='treatment',
    data=df, ax=ax[1,0],
    palette='icefire'
)
ax[1,0].set_xlabel("Bar's threshold")
ax[1,0].set_ylabel("Alaternation Index")
ax[1,0].set_title("Alternation index per threshold")
ax[1,0].grid()
# ax[1,0].set_ylim([-1,1])
ax[1,0].legend().remove()

# Alternation index per number of agents and treatment
sns.lineplot(
    x='num_players', y='alternation_index',
    hue='treatment',
    data=df, ax=ax[1,1],
    palette='icefire'
)
ax[1,1].set_xlabel("Number of players")
ax[1,1].set_ylabel("Alaternation Index")
ax[1,1].set_title("Alternation index per Num. Players")
ax[1,1].grid()
# ax[1,1].set_ylim([-1,1])
ax[1,1].legend().remove()

# Convert figure to image array
handles, labels = ax[0,0].get_legend_handles_labels()
fig.legend(
    handles, labels,
    title='Treatment',
    loc='lower center', 
    bbox_to_anchor=(.5, -.1),
    ncol=2
)
fig.suptitle("E", fontweight="bold")
# fig.canvas.draw()
# rgba_bytes = fig.canvas.buffer_rgba().tobytes()
# width, height = fig.canvas.get_width_height()
# pil_image = PIL.Image.frombytes(mode="RGBA", size=[width, height], data=rgba_bytes)
# fourth_row_array = np.array(pil_image)

buf = BytesIO()
fig.savefig(buf, format='png', bbox_inches='tight', dpi=fig.dpi)
buf.seek(0)

# Read the buffer into a NumPy array
pil_image = Image.open(buf).convert("RGBA")
panel_E = np.array(pil_image)


# All panels for Figure 12

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(6,9), tight_layout=True)

ax[0].imshow(panels_A_and_B)
ax[1].imshow(panels_C_and_D)
ax[2].imshow(panel_E)

ax[0].axis('off')
ax[1].axis('off') 
ax[2].axis('off')



In [ ]:
fig.savefig(PATHS['figures_for_paper'] / 'Figure12.png', dpi=300, bbox_inches='tight')

### Regressions

In [ ]:
formula = 'score ~ treatment + num_players + threshold + round + session ' 
formula += '+ num_players * threshold + round * session + treatment * threshold + treatment * num_players'
lm = ols(formula, data).fit()

print(lm.summary())

In [ ]:
# Calculate alternation index per group
gm = GetMeasurements(
    data=data,
    measures=['efficiency', 'bounded_efficiency', 'inequality'],
)
df = gm.get_measurements()

alt_index = AlternationIndex.from_file(priority='sklearn')
df['alternation_index'] = alt_index(df)

df.head(2)

In [ ]:
formula = 'bounded_efficiency ~ treatment + num_players + threshold'
formula += '+ num_players * threshold + treatment * threshold + treatment * num_players'
lm = ols(formula, df).fit()

print(lm.summary())

In [ ]:
formula = 'alternation_index ~ treatment + num_players + threshold'
formula += '+ num_players * threshold + treatment * threshold + treatment * num_players'
lm = ols(formula, df).fit()

print(lm.summary())

### Check collective action dilemma

In [ ]:
player_av_scores = data.groupby(['player']).agg({
    'score':'mean',
    'decision':'mean'
})

group_av_scores = data.groupby(['group']).agg({
    'score':'mean',
    'decision':'mean'
})

In [ ]:
from scipy.stats import pearsonr

In [ ]:
r, p_value = pearsonr(player_av_scores['decision'], player_av_scores['score'])

print(f"Pearson correlation coefficient: {r:.4f}")
print(f"P-value: {p_value:.4e}")
print(f"{r:.3f} (p={p_value:.3f})")

In [ ]:
r, p_value = pearsonr(group_av_scores['decision'], group_av_scores['score'])

print(f"Pearson correlation coefficient: {r:.4f}")
print(f"P-value: {p_value:.4e}")
print(f"{r:.3f} (p={p_value:.3f})")

---